# sistema de assistência virtual do zero

# 1. Instalação de dependências e imports
Instalamos as bibliotecas necessárias para **reconhecimento de voz, síntese de fala e pesquisa na Wikipédia**:  

- `SpeechRecognition` → reconhecimento de áudio.  
- `pydub` → manipulação de arquivos de áudio.  
- `gTTS` → conversão de texto para fala.  
- `wikipedia` → busca de informações na Wikipédia.  

In [ ]:
!pip install SpeechRecognition pydub
!pip install gTTS wikipedia

import os
import time
from gtts import gTTS
from IPython.display import display, Audio, HTML
import wikipedia
import speech_recognition as sr
from pydub import AudioSegment

## 2. Função para síntese de fala
Criamos uma função que converte **texto em áudio** usando `gTTS` e exibe um **player de áudio interativo** no notebook:  

- Incrementa um contador para gerar arquivos de áudio únicos (`voz_1.mp3`, `voz_2.mp3`, ...).  
- Aguarda o arquivo ser salvo antes de exibir o player.  
- Permite reproduzir a fala diretamente no notebook.  

In [ ]:
contador_audio = 0
def falar(texto):
    global contador_audio
    contador_audio += 1
    nome_arquivo = f"voz_{contador_audio}.mp3"

    # Gera o arquivo de áudio
    tts = gTTS(text=texto, lang="pt")
    tts.save(nome_arquivo)

    # Garante que o arquivo foi salvo
    while not os.path.exists(nome_arquivo):
        time.sleep(0.1)

    # Mensagem clara para o usuário clicar
    display(HTML(f"<b>Clique no play para ouvir:</b>"))

    # Exibe player de áudio
    display(Audio(nome_arquivo))

falar("Bom dia")

## 3. Funções de conversão de áudio e transcrição
Implementamos funções para **converter MP3 para WAV** e **transformar áudio em texto** usando `SpeechRecognition`:  

- `mp3_para_wav` → converte arquivos `.mp3` para `.wav` para compatibilidade com o reconhecedor.  
- `audio_para_texto` → recebe um arquivo de áudio e retorna o **texto transcrito** em português.  
- `Tratamento de exceções`: áudio não reconhecido ou erro no serviço Google

In [ ]:
def mp3_para_wav(mp3_arquivo, wav_arquivo):
    audio = AudioSegment.from_mp3(mp3_arquivo)
    audio.export(wav_arquivo, format="wav")

# Função para transformar áudio em texto
def audio_para_texto(arquivo_audio):
    r = sr.Recognizer()

    # Converte MP3 para WAV se necessário
    if arquivo_audio.endswith(".mp3"):
        wav_arquivo = arquivo_audio.replace(".mp3", ".wav")
        mp3_para_wav(arquivo_audio, wav_arquivo)
    else:
        wav_arquivo = arquivo_audio

    with sr.AudioFile(wav_arquivo) as source:
        audio = r.record(source)
        try:
            texto = r.recognize_google(audio, language="pt-BR")
            return texto
        except sr.UnknownValueError:
            return "Não foi possível reconhecer o áudio."
        except sr.RequestError as e:
            return f"Erro no serviço de reconhecimento: {e}"

nome_arquivo = f"voz_{1}.mp3"
if os.path.exists(nome_arquivo):
    texto = audio_para_texto(nome_arquivo)
    print(f"Arquivo {nome_arquivo} transcrito: {texto}")
else:
    print(f"Arquivo {nome_arquivo} não encontrado.")


Arquivo voz_1.mp3 transcrito: Bom dia


## 4. Assistente virtual interativo
Implementamos um **assistente virtual** que recebe comandos do usuário e responde em **áudio**:  

- `ouvir_comando()` → solicita o comando do usuário via `input` e reproduz o texto em áudio.  
- `executar_comando()` → executa ações com base no comando:  
  - **Wikipedia** → pesquisa e retorna resumo em português, com link clicável.  
  - **YouTube** → gera link de busca no YouTube e fala a resposta.  
  - **Farmácia / Localização** → mostra farmácias próximas via Google Maps.  
  - Comandos não reconhecidos → aviso em áudio.  
- Loop principal → mantém o assistente ativo até o usuário digitar `"sair"`


In [ ]:

# Função: recebe comando do usuário e lê em áudio
def ouvir_comando():
    comando = input("Digite seu comando (ou 'sair' para encerrar): ")
    comando_lower = comando.lower()  # só para comparar comandos
    falar(f"Você digitou: {comando}")
    return comando, comando_lower

# Função: executa comando
def executar_comando(comando, comando_lower):
    if "wikipedia" in comando_lower:
        consulta = input("Digite o que deseja pesquisar na Wikipedia: ")
        falar(f"Você digitou: {consulta}")

        wikipedia.set_lang("pt")  # define o idioma
        resultados = wikipedia.search(consulta)  # pesquisa aproximada

        if resultados:
            titulo = resultados[0]  # pega o primeiro resultado
            try:
                resumo = wikipedia.summary(titulo, sentences=3)  # resumo curto
                url = wikipedia.page(titulo).url  # pega link da página
                falar(f"Resumo da Wikipedia sobre {titulo}: {resumo}")
                display(HTML(f'<a href="{url}" target="_blank">Abrir página da Wikipedia: {titulo}</a>'))
            except wikipedia.exceptions.DisambiguationError as e:
                falar(f"O termo '{consulta}' é ambíguo. Algumas opções são: {', '.join(e.options[:5])}")
        else:
            falar("Não encontrei nada na Wikipedia.")

    elif "youtube" in comando_lower:
        consulta = input("Digite o que deseja pesquisar no YouTube: ")
        falar(f"Você digitou: {consulta}")
        url = f"https://www.youtube.com/results?search_query={consulta.replace(' ', '+')}"
        display(HTML(f'<a href="{url}" target="_blank">Abrir no YouTube: {consulta}</a>'))
        falar(f"Aqui está o link para {consulta} no YouTube.")

    elif "farmácia" or "farmacia" in comando_lower or "localização" in comando_lower:
        url = "https://www.google.com/maps/search/farmácia+perto+de+mim/"
        display(HTML(f'<a href="{url}" target="_blank">Abrir farmácias próximas</a>'))
        falar("Mostrando farmácias próximas no mapa.")

    else:
        falar("Comando não reconhecido.")

# Loop principal
falar("Olá! Eu sou seu assistente virtual. Como posso ajudar?")
while True:
    comando, comando_lower = ouvir_comando()
    if comando_lower == "sair":
        falar("Encerrando o assistente. Até mais!")
        break
    executar_comando(comando, comando_lower)




Digite seu comando (ou 'sair' para encerrar): wikipedia


Digite o que deseja pesquisar na Wikipedia: gato


Digite seu comando (ou 'sair' para encerrar): farmacia


Digite seu comando (ou 'sair' para encerrar): youtube


Digite o que deseja pesquisar no YouTube: dust in the wind


Digite seu comando (ou 'sair' para encerrar): sair


## 5. Transcrever todos os arquivos MP3
Percorremos todos os arquivos de áudio gerados pelo assistente e transformamos em **texto** usando a função `audio_para_texto`:  

- Verifica se o arquivo existe.  
- Converte e transcreve o áudio.  
- Exibe o texto resultante no notebook.  



In [ ]:
#  transforma todos os arquivos MP3 em texto

for i in range(1, contador_audio + 1):
    nome_arquivo = f"voz_{i}.mp3"
    if os.path.exists(nome_arquivo):
        texto = audio_para_texto(nome_arquivo)
        print(f"Arquivo {nome_arquivo} transcrito: {texto}")
    else:
        print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo voz_1.mp3 transcrito: Bom dia
Arquivo voz_2.mp3 transcrito: Olá eu sou seu assistente virtual como posso ajudar
Arquivo voz_3.mp3 transcrito: você digitou Wikipédia
Arquivo voz_4.mp3 transcrito: você digitou gato
Arquivo voz_5.mp3 transcrito: resumo da Wikipédia sobre gato o gato nome científico felis catus o gato doméstico é um mamífero carnívoro da família dos felídeos muito popular como animal de estimação ocupando o topo da cadeia alimentar é predador natural de diversos animais como roedores pássaros lagartixas e alguns insetos segundo pesquisas realizadas por instituições norte-americanas os gatos consistem no segundo animal de estimação mais popular do mundo estando numericamente atrás apenas dos peixes de aquário
Arquivo voz_6.mp3 transcrito: você digitou farmácia
Arquivo voz_7.mp3 transcrito: mostrando farmácias próximas no mapa
Arquivo voz_8.mp3 transcrito: você digitou YouTube
Arquivo voz_9.mp3 transcrito: você digitou Dash In The Wind
Arquivo voz_10.mp3 transcrito: 